<a href="https://colab.research.google.com/github/saurabh-maurya/Human-or-Horse-using-ConvNet/blob/master/horse_or_human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-06-06 14:58:05--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M   120MB/s    in 1.2s    

2020-06-06 14:58:07 (120 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2020-06-06 14:58:10--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  36.8MB/s    in 0.3s    

2020-06-06 14:58:10 (36.8 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



In [0]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

# Image Generator
 - Firstly, create dir **Images** with sub-dir **Training** and **Validation** with sub-dir **Horses** and **Humans** in each sub-dir.


```
Images --> Training --> Horses
                    --> Humans

       --> Validation --> Horses
                      --> Humans

```





In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
train_dir = '/tmp/horse-or-human/'
validation_dir = '/tmp/validation-horse-or-human/'

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (300, 300),
    batch_size = 128,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [0]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size = (300, 300),
    batch_size = 32,
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


# Model Building

In [0]:
import tensorflow as tf

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 33, 33, 64)       

In [0]:
from tensorflow.keras.optimizers import RMSprop

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

In [19]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 7s 849ms/step - loss: 1.2999 - accuracy: 0.4950 - val_loss: 0.6782 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 8s 973ms/step - loss: 0.6529 - accuracy: 0.6029 - val_loss: 0.7140 - val_accuracy: 0.5000
Epoch 3/15
8/8 [==============================] - 7s 861ms/step - loss: 0.5872 - accuracy: 0.7197 - val_loss: 0.6333 - val_accuracy: 0.5117
Epoch 4/15
8/8 [==============================] - 7s 852ms/step - loss: 0.5621 - accuracy: 0.7230 - val_loss: 0.5338 - val_accuracy: 0.8438
Epoch 5/15
8/8 [==============================] - 7s 851ms/step - loss: 0.3340 - accuracy: 0.8765 - val_loss: 0.6300 - val_accuracy: 0.8516
Epoch 6/15
8/8 [==============================] - 7s 857ms/step - loss: 0.2227 - accuracy: 0.9221 - val_loss: 1.3315 - val_accuracy: 0.7930
Epoch 7/15
8/8 [==============================] - 7s 847ms/step - loss: 0.0645 - accuracy: 0.9766 - val_loss: 0.7629 - val_accuracy: 0.8945
Epoch 8/15
8/8 [====

# Running the Model 
- upload image for prediction using GUI

In [20]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving horse.jpg to horse (1).jpg
Saving Iron man.jpg to Iron man (1).jpg
[0.]
horse.jpg is a horse
[1.]
Iron man.jpg is a human


## Clean Up
Run the following cell to terminate the kernel and free memory resources

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)